GooglePalm LLM and API setup

In [14]:
from langchain.llms import GooglePalm
from API_KEYS import MakerSuite_api_key

In [18]:
api_key = MakerSuite_api_key

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

/home/mohamedaliabid/anaconda3/envs/Retail/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connect with the database

In [21]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [22]:
db_user = "mohamedaliabid10"
db_password = "messi275806501010"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

Question 1 - Good answer

In [40]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qs1 =db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")
print(qs1)



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(69,)]
Answer:69
> Finished chain.
69


In [31]:
#We notice that the LLM model can interpret that extra small size = xs and it transfer th white color into Capital "W" and thats because the LLM
#model looked into the database and figured that out color starts with capital "W".
#Some notes : 
#Word embeddings play a crucial role in how language models like GPT-3.5 understand and interpret language. Word embeddings are vector representations of words in a continuous vector space, where words with similar meanings are mapped to similar vectors.
#During training, the model learns these embeddings by analyzing the context in which words appear in a vast amount of text data. In your case, the model has likely encountered numerous instances where phrases like "extra small" co-occur with the size 'XS' in the context of clothing descriptions or queries.
#So, when you provided the prompt mentioning "extra small size," the model was able to leverage its learned embeddings to associate "extra small" with the specific size representation 'XS'. It generalizes from the patterns it learned during training to make predictions or generate responses when faced with similar language constructs.
#This ability to capture and utilize semantic relationships between words is a key strength of word embeddings and contributes to the model's capacity to understand and generate human-like text.

Question 2 - Wrong answer

In [41]:
qs2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('246'),)]
Answer:246
> Finished chain.


In [ ]:
#It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names.
#We need to tell it somehow that price is price per unit and the actual query should be,
#SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [42]:
qs2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('13811'),)]
Answer:13811
> Finished chain.


Question 3 -Wrong aswear

In [43]:
qs3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [69]:
#it returned a wrong query which generated an error during query execution. It thinks discount table would have start and end date which is normally true but in our table there is no start or end date column. 
#One thing we can do here is run the query directly.

In [44]:
sql_code = """
SELECT 
        SUM((`a`.`total_amount` * ((100 - COALESCE(`discounts`.`pct_discount`, 0)) / 100))) AS `total_revenue`
    FROM
        ((SELECT 
            SUM((`t_shirts`.`price` * `t_shirts`.`stock_quantity`)) AS `total_amount`,
                `t_shirts`.`t_shirt_id` AS `t_shirt_id`
        FROM
            `t_shirts`
        WHERE
            (`t_shirts`.`brand` = 'Levi')
        GROUP BY `t_shirts`.`t_shirt_id`) `a`
        LEFT JOIN `discounts` ON ((`a`.`t_shirt_id` = `discounts`.`t_shirt_id`)))
 """

qs3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

SELECT 
        SUM((`a`.`total_amount` * ((100 - COALESCE(`discounts`.`pct_discount`, 0)) / 100))) AS `total_revenue`
    FROM
        ((SELECT 
            SUM((`t_shirts`.`price` * `t_shirts`.`stock_quantity`)) AS `total_amount`,
                `t_shirts`.`t_shirt_id` AS `t_shirt_id`
        FROM
            `t_shirts`
        WHERE
            (`t_shirts`.`brand` = 'Levi')
        GROUP BY `t_shirts`.`t_shirt_id`) `a`
        LEFT JOIN `discounts` ON ((`a`.`t_shirt_id` = `discounts`.`t_shirt_id`)))
 
SQLQuery:SELECT 
       SUM((`a`.`total_amount` * ((100 - COALESCE(`discounts`.`pct_discount`, 0)) / 100))) AS `total_revenue`
   FROM
       ((SELECT 
           SUM((`t_shirts`.`price` * `t_shirts`.`stock_quantity`)) AS `total_amount`,
               `t_shirts`.`t_shirt_id` AS `t_shirt_id`
       FROM
           `t_shirts`
       WHERE
           (`t_shirts`.`brand` = 'Levi')
       GROUP BY `t_shirts`.`t_shirt_id`) `a`
       LEFT JOIN `d

Question 4

In [45]:
qs4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('9223'),)]
Answer:9223
> Finished chain.


Question 5 - Wrong answer

In [46]:
qs5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(62,), (95,), (42,)]
Answer:42
> Finished chain.


In [38]:
#Wrong answer . 
#We need to use SUM(stock_quantity).

In [47]:
qs5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('199'),)]
Answer:199
> Finished chain.


Few shot learning

In [48]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qs1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qs2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qs3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qs4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qs5
     }
]

Semantic Similarity

* create embedding on the few_shots
* Store the embeddings in Chroma DB
* Retrieve the the top most Semantically close example from the vector store

In [59]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [52]:
# Word embeddings models in sentence-transformers : (link)
# https://huggingface.co/sentence-transformers?sort_models=downloads#models

In [57]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [53]:
e = embeddings.embed_query("How much is the total price of the inventory for all S-size t-shirts?")

In [56]:
e[:5]

[0.01736629568040371,
 0.04344544559717178,
 -0.024143826216459274,
 0.026405828073620796,
 0.03335282579064369]

In [ ]:
#The result shows list of number . the number dont actually make sense but they capture the meaning of this particular promt in the right way.
#so that if someone write similar promt or different promt similar to that promt but the words are different even , the embedding of the new promt
#would be similar to this promt in terms of COSINE similarity.
#exmple if someone write this promt : How much is the total price of the inventory for all S-size t-shirts?
#we will get the correct outcome .

In [58]:
vectorize = [" ".join(example.values()) for example in few_shots]
vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 69",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 13811",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 9012.4",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' R

In [68]:
#This link is to learn more about from langchain.vectorstores.Chroma
#https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.chroma.Chroma.html#langchain.vectorstores.chroma.Chroma.from_texts

In [60]:
vector_db = Chroma.from_texts(vectorize, embeddings, metadatas=few_shots)

In [67]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_db,
    k=2,
)

example_selector.select_examples({"Question": "How many Levi white T shirts I have left in my store?"})

[{'Answer': '69',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '199',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [70]:
#in question 3 ,the LLM thinks discount table would have start and end date which is normally true but in our table there is no start or end date column. 
#so we need to setup PromptTemplate so when the model get confused it will return to the template .

PromptTemplete using input variables

In [71]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [72]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [73]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [74]:
from langchain.prompts.prompt import PromptTemplate

In [75]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [79]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [80]:
print(mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLRes

In [81]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #the prefix and suffix
)

In [82]:
final_db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [83]:
final_db_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('199'),)]
Answer:199
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '199'}

its now giving us the correct result !!